In [1]:
import numpy as np
import open3d as o3d
import pandas as pd
from sklearn.cluster import KMeans
import random
import os
import glob
from open3d.web_visualizer import draw

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
def preprocess_point_cloud(pcd):
    pcd = pcd.voxel_down_sample(voxel_size=0.02)

    radius_normal = 0.2
    print("estimating normals")
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = 0.5
    print("computing fpfh")
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    fpfh = np.asarray(pcd_fpfh.data).T
    return pcd, fpfh

In [3]:
def cov_metrics(pcd):
    print("estimating covariances")
    covs = pcd.estimate_covariances()
    covs = np.asarray(pcd.covariances)

    print("computing eigen-based metrics")
    metrics = []
    for pt in covs:
        eigenvalues = np.linalg.eigvals(pt)
        e1, e2, e3 = eigenvalues
        
        linearity = (e1 - e2) / e1
        planarity = (e2 - e3) / e1
        scattering = e3 / e1
        omnivariance = (e1 * e2 * e3) ** (1 / 3)
        anisotropy = (e1 - e3) / e1
        eigentropy = -(e1 * np.log(e1) + e2 * np.log(e2) + e3 * np.log(e3))
        curvature = e3 / (e1 + e2 + e3)

        metrics.append((linearity, planarity, scattering, omnivariance, anisotropy, eigentropy, curvature))

    dtype = [('linearity', 'f8'), ('planarity', 'f8'), ('scattering', 'f8'), 
            ('omnivariance', 'f8'), ('anisotropy', 'f8'), ('eigentropy', 'f8'), 
            ('curvature', 'f8')]
    
    metrics_array = np.array(metrics, dtype=dtype)
  
    return np.array([tuple(row) for row in metrics_array])

In [4]:
def prep_pcd(pcd_path):
    pcd = o3d.io.read_point_cloud(pcd_path)
    pcd, fpfh = preprocess_point_cloud(pcd)
    cov = cov_metrics(pcd)
    
    cov_headers = ['linearity', 'planarity', 'scattering', 'omnivariance', 'anisotropy', 'eigentropy', 'curvature']
    header = ['x', 'y', 'z'] + [f'feature{i}' for i in range(fpfh.shape[1])] + cov_headers
    all_metrics = np.hstack([np.asarray(pcd.points), fpfh, cov])

    metrics = pd.DataFrame(all_metrics, columns=header)
    #metrics = metrics[metrics['z'] <= 0.25]
    return pcd, metrics

In [5]:
def process_multiple(folder_path):
    all_metrics = []

    paths = glob.glob(f"{folder_path}/**/*.pcd", recursive=True)
    paths = [os.path.normpath(path).replace(os.sep, '/') for path in paths]

    for file_ct, path in enumerate(paths, start=1):
        pcd, metrics = prep_pcd(path)
        metrics["scan_num"] = file_ct
        all_metrics.append(metrics)

    return all_metrics

In [6]:
scans = process_multiple("C:/Users/ellie/OneDrive/Desktop/lidar_local/ccb_no_trees")

estimating normals
computing fpfh
estimating covariances
computing eigen-based metrics
estimating normals
computing fpfh
estimating covariances
computing eigen-based metrics
estimating normals
computing fpfh
estimating covariances
computing eigen-based metrics


In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

scans_full = np.vstack((scans[0], scans[1]))
scans_full = np.vstack((scans_full, scans[2]))

scans_full_aboveZ = scans_full#[scans_full[:, 2] > 0.025]
print(pd.DataFrame(scans_full_aboveZ).shape)
coords = scans_full_aboveZ[:, :3]
print(pd.DataFrame(coords, columns=['x','y','z']).shape)
coords_scan = np.concatenate([coords, scans_full_aboveZ[:, -1].reshape(-1, 1)], axis=1)
features = scans_full_aboveZ[:, 3:]
print(features.shape)

scaled_features = pipeline.fit_transform(features)
print(scaled_features.shape)

(3106080, 44)
(3106080, 3)
(3106080, 41)
(3106080, 41)


In [8]:
from sklearn.cluster import KMeans

coords_scan_df = pd.DataFrame(coords_scan, columns=['x','y','z', 'scan'])
ks = [2, 3, 4, 5, 6, 7]
errors = {}
min_inertia = 2**1000
optimal_k = 0
for k in ks:
    print(k)
    kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(scaled_features)
    errors[k] = kmeans.inertia_
    if kmeans.inertia_ < min_inertia:
        min_inertia = kmeans.inertia_
        optimal_k = k
kmeans = KMeans(n_clusters=optimal_k, random_state=0, n_init="auto").fit(scaled_features)
coords_scan_df['cluster'] = kmeans.labels_
print(coords_scan_df)

2
3
4


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
ks = list(errors.keys())
sse_values = list(errors.values())
plt.figure(figsize=(8, 5))
plt.plot(ks, sse_values, marker='o', linestyle='-', color='b')
plt.show()

In [11]:
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(scaled_features)
coords_scan_df['cluster'] = kmeans.labels_

In [13]:
scan_1 = coords_scan_df[np.isclose(coords_scan_df.iloc[:, 3], 1.0)]
just_clusters = scan_1[["x", "y", "z", "cluster"]]
#just_clusters = just_clusters[just_clusters['z']<0.3]

for i in range(5):  # Handle up to 5 clusters or the number of found clusters
    cluster_points = just_clusters[just_clusters['cluster'] == i][['x', 'y', 'z']].values
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cluster_points)
    o3d.visualization.draw_geometries([pcd])

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=10, min_samples=100000).fit(scaled_features)
coords_scan_df['cluster'] = dbscan.labels_
scan_1 = coords_scan_df[np.isclose(coords_scan_df.iloc[:, 3], 3.0)]
just_clusters = scan_1[["x", "y", "z", "cluster"]]
#just_clusters = just_clusters[just_clusters['z']<0.3]

for i in range(5):  # Handle up to 5 clusters or the number of found clusters
    cluster_points = just_clusters[just_clusters['cluster'] == i][['x', 'y', 'z']].values
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cluster_points)
    o3d.visualization.draw_geometries([pcd])